In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [2]:
import numpy as np

In [3]:
import tensorflow as tf

In [4]:
tf.logging.set_verbosity(tf.logging.INFO)

In [ ]:
def cnn_model_fn(features,labels,mode):
    input_layer = tf.reshape(features['x'],[-1,28,28,1])
    
    conv1 = tf.layers.conv2d(inputs=input_layer,filters=32,kernel_size=[5,5],padding='same',activation=tf.nn.relu)
    pool1 = tf.layers.max_pooling2d(inputs=conv1,pool_size=[2,2],strides=2)
    
    conv2 = tf.layers.conv2d(inputs=pool1,filters=64,kernel_size=[5,5],padding='same',activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2,pool_size=[2,2],strides=2)
    
    pool2_flat = tf.reshape(pool2,[-1,7*7*64])
    dense = tf.layers.dense(inputs=pool2_flat,units=1024,activation=tf.nn.relu)
    dropout = tf.layers.dropout(inputs=dense,rate=0.4,training=mode==tf.estimator.ModeKeys.TRAIN)
    
    logits = tf.layers.dense(inputs=dropout,units=10)
    
    predictions = {
        
        'classes': tf.argmax(input=logits,axis=1),
        'probabilities': tf.nn.softmax(logits,name='softmax_tensor')       
        
    }
    
    loss = tf.losses.spares_softmax_cross_entropy(labels=labels,logits=logits)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(loss =loss
                                     global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode,loss=loss,train_op=train_op)
    
    eval_metric_ops = {
        'accuracy': tf.metrics.accuracy(
        labels=labels,predictions=predictions['classes'])
    }
    return tf.estimator.EstimatorSpec(
        mode=mode,loss=loss, eval_metric_ops=eval_metric_ops)
        